# Stock Price Projection Model 

## Seting Up enviroment 

In [1]:
# Cell 1: Install required libraries (run once)
!pip install -q yfinance pandas numpy matplotlib seaborn scikit-learn statsmodels tensorflow datasets ta

# Notes:
# - 'ta' is a lightweight technical analysis library (alternative to TA-Lib).
# - 'datasets' is the Hugging Face library to load their datasets.
# - Use '-q' to suppress verbose output.

In [2]:
# Cell 2: Import libraries and set up logging
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import os
import sys
from datetime import datetime
from datasets import load_dataset, DatasetDict
import tensorflow as tf

# Set up logging to file and console
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("stock_project.log"),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

# Create directories for data and models
os.makedirs("data/raw", exist_ok=True)
os.makedirs("data/processed", exist_ok=True)
os.makedirs("models", exist_ok=True)

# Set random seeds for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

logger.info("Environment setup complete.")

/home/duvan/Documents/StockPredictionModel/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-02-17 13:35:51.918320: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-17 13:35:52.188029: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-17 13:35:53.884286: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


2026-02-17 13:35:54,227 - INFO - Environment setup complete.


In [3]:
# Cell 3: Configuration parameters
TICKER = "NVDA"
START_DATE = "2010-01-01"
END_DATE = "2025-12-31"
VIX_TICKER = "^VIX"
WINDOW_SIZE = 60          # for sequence creation
TEST_SIZE = 0.2            # proportion of data for testing (if not using date split)
VAL_SIZE = 0.1             # proportion of training data for validation

# Paths
RAW_DATA_PATH = "data/raw"
PROCESSED_DATA_PATH = "data/processed"

logger.info(f"Configuration set for {TICKER} from {START_DATE} to {END_DATE}")

2026-02-17 13:35:54,235 - INFO - Configuration set for NVDA from 2010-01-01 to 2025-12-31


## Data Acquisition

In [4]:
# Cell 4: Download NVIDIA data from Yahoo Finance
logger.info(f"Downloading {TICKER} data from Yahoo Finance...")
try:
    nvda_yf = yf.download(TICKER, start=START_DATE, end=END_DATE, progress=False)
    logger.info(f"Downloaded {len(nvda_yf)} rows from Yahoo Finance.")
except Exception as e:
    logger.error(f"Failed to download {TICKER}: {e}")
    nvda_yf = pd.DataFrame()

# Download VIX (volatility index) as an additional feature
logger.info("Downloading VIX data...")
try:
    vix = yf.download(VIX_TICKER, start=START_DATE, end=END_DATE, progress=False)['Adj Close']
    vix.name = 'VIX'
    logger.info(f"Downloaded {len(vix)} rows of VIX.")
except Exception as e:
    logger.error(f"Failed to download VIX: {e}")
    vix = pd.Series(dtype=float)

# Save raw Yahoo data
if not nvda_yf.empty:
    nvda_yf.to_csv(os.path.join(RAW_DATA_PATH, f"{TICKER}_yf.csv"))
    vix.to_csv(os.path.join(RAW_DATA_PATH, "VIX.csv"))
    logger.info("Raw Yahoo data saved.")

2026-02-17 13:35:54,244 - INFO - Downloading NVDA data from Yahoo Finance...
2026-02-17 13:35:56,001 - INFO - Downloaded 4023 rows from Yahoo Finance.
2026-02-17 13:35:56,002 - INFO - Downloading VIX data...
2026-02-17 13:35:56,742 - ERROR - Failed to download VIX: 'Adj Close'
2026-02-17 13:35:56,766 - INFO - Raw Yahoo data saved.


In [5]:
# Cell 5: Load MTBench dataset (high-frequency 5-min data)
logger.info("Loading MTBench stock dataset from Hugging Face...")
# This will download the dataset (approx 2-3 GB). It may take a while.
try:
    mtbench_dataset = load_dataset("afeng/MTBench_finance_stock")
    logger.info(f"MTBench dataset loaded. Structure: {mtbench_dataset}")
    # Save dataset info for later reference
    with open(os.path.join(RAW_DATA_PATH, "mtbench_info.txt"), "w") as f:
        f.write(str(mtbench_dataset))
except Exception as e:
    logger.error(f"Failed to load MTBench: {e}")
    mtbench_dataset = None

2026-02-17 13:35:56,774 - INFO - Loading MTBench stock dataset from Hugging Face...
2026-02-17 13:35:57,312 - INFO - HTTP Request: HEAD https://huggingface.co/datasets/afeng/MTBench_finance_stock/resolve/main/README.md "HTTP/1.1 307 Temporary Redirect"
2026-02-17 13:35:57,415 - INFO - HTTP Request: HEAD https://huggingface.co/datasets/GGLabYale/MTBench_finance_stock/resolve/main/README.md "HTTP/1.1 307 Temporary Redirect"
2026-02-17 13:35:57,510 - INFO - HTTP Request: HEAD https://huggingface.co/api/resolve-cache/datasets/GGLabYale/MTBench_finance_stock/0c1a656a9611846e76c35a96ec85404829763022/README.md "HTTP/1.1 200 OK"
2026-02-17 13:35:57,624 - INFO - HTTP Request: HEAD https://huggingface.co/datasets/afeng/MTBench_finance_stock/resolve/0c1a656a9611846e76c35a96ec85404829763022/MTBench_finance_stock.py "HTTP/1.1 307 Temporary Redirect"
2026-02-17 13:35:57,729 - INFO - HTTP Request: HEAD https://huggingface.co/datasets/GGLabYale/MTBench_finance_stock/resolve/0c1a656a9611846e76c35a96ec8

Repo card metadata block was not found. Setting CardData to empty.


2026-02-17 13:35:58,250 - WARNING - Repo card metadata block was not found. Setting CardData to empty.
2026-02-17 13:35:58,432 - INFO - HTTP Request: GET https://huggingface.co/api/datasets/afeng/MTBench_finance_stock/revision/0c1a656a9611846e76c35a96ec85404829763022 "HTTP/1.1 307 Temporary Redirect"
2026-02-17 13:35:58,546 - INFO - HTTP Request: GET https://huggingface.co/api/datasets/GGLabYale/MTBench_finance_stock/revision/0c1a656a9611846e76c35a96ec85404829763022 "HTTP/1.1 200 OK"
2026-02-17 13:35:58,657 - INFO - HTTP Request: HEAD https://huggingface.co/datasets/afeng/MTBench_finance_stock/resolve/0c1a656a9611846e76c35a96ec85404829763022/.huggingface.yaml "HTTP/1.1 307 Temporary Redirect"
2026-02-17 13:35:58,783 - INFO - HTTP Request: HEAD https://huggingface.co/datasets/GGLabYale/MTBench_finance_stock/resolve/0c1a656a9611846e76c35a96ec85404829763022/.huggingface.yaml "HTTP/1.1 404 Not Found"
2026-02-17 13:35:59,251 - INFO - HTTP Request: GET https://datasets-server.huggingface.co/

In [ ]:
# Cell 6: Extract NVIDIA data from MTBench (if available)
if mtbench_dataset is not None:
    # The MTBench dataset structure: we need to filter for NVDA.
    # Usually, it's a DatasetDict with 'train' split containing all stocks.
    # We'll assume the 'train' split has a 'ticker' column.
    try:
        # Convert to pandas for easier filtering (be mindful of memory)
        df_mtbench = mtbench_dataset['train'].to_pandas()
        nvda_mtbench = df_mtbench[df_mtbench['ticker'] == 'NVDA'].copy()
        logger.info(f"Extracted {len(nvda_mtbench)} rows for NVDA from MTBench.")
        # Save to CSV
        nvda_mtbench.to_csv(os.path.join(RAW_DATA_PATH, "NVDA_MTBench.csv"), index=False)
    except Exception as e:
        logger.error(f"Failed to extract NVDA from MTBench: {e}")
        nvda_mtbench = pd.DataFrame()

In [ ]:
# Cell 7: Load FNSPID dataset (news + prices)
logger.info("Loading FNSPID dataset (this is large, may take time)...")
try:
    # The FNSPID dataset is available at https://huggingface.co/datasets/Zihan1004/FNSPID
    # It contains both price and news data. We'll load the full dataset.
    fnspid_dataset = load_dataset("Zihan1004/FNSPID")
    logger.info(f"FNSPID loaded. Structure: {fnspid_dataset}")
    with open(os.path.join(RAW_DATA_PATH, "fnspid_info.txt"), "w") as f:
        f.write(str(fnspid_dataset))
except Exception as e:
    logger.error(f"Failed to load FNSPID: {e}")
    fnspid_dataset = None

In [ ]:
# Cell 8: (Optional) Load NVIDIA-specific fundamental dataset
logger.info("Loading NVIDIA fundamental dataset from Hugging Face...")
try:
    nvda_fundamental = load_dataset("imdad19/Nvidia_data")
    logger.info(f"Fundamental dataset loaded: {nvda_fundamental}")
    # Convert to DataFrame and save
    df_fund = nvda_fundamental['train'].to_pandas()
    df_fund.to_csv(os.path.join(RAW_DATA_PATH, "NVDA_fundamental.csv"), index=False)
except Exception as e:
    logger.error(f"Failed to load fundamental dataset: {e}")

In [ ]:
# Cell 9: Quick look at the downloaded data
logger.info("Sample of Yahoo Finance data:")
display(nvda_yf.head())

if not nvda_mtbench.empty:
    logger.info("Sample of MTBench data for NVDA:")
    display(nvda_mtbench.head())

logger.info("Phase 1 complete. All raw data saved in 'data/raw/'.")

## Exploratory Data Analysis 

In [ ]:
# Cell 10: Import additional libraries for EDA (if not already imported)
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)

logger.info("Starting Phase 2: Exploratory Data Analysis")

In [ ]:
# Cell 11: Load the Yahoo Finance data (saved earlier)
nvda_yf = pd.read_csv(os.path.join(RAW_DATA_PATH, f"{TICKER}_yf.csv"), index_col=0, parse_dates=True)
vix = pd.read_csv(os.path.join(RAW_DATA_PATH, "VIX.csv"), index_col=0, parse_dates=True, squeeze=True)

# Ensure datetime index
nvda_yf.index = pd.to_datetime(nvda_yf.index)
vix.index = pd.to_datetime(vix.index)

logger.info(f"Data loaded: {nvda_yf.shape[0]} rows, {nvda_yf.shape[1]} columns")
logger.info(f"Date range: {nvda_yf.index.min()} to {nvda_yf.index.max()}")

In [ ]:
# Cell 12: Basic info and missing values
print("Data types:")
print(nvda_yf.dtypes)
print("\nMissing values:")
print(nvda_yf.isnull().sum())

# Plot missing values (if any)
if nvda_yf.isnull().sum().sum() > 0:
    plt.figure(figsize=(10, 4))
    sns.heatmap(nvda_yf.isnull(), cbar=False, yticklabels=False)
    plt.title("Missing Values Heatmap")
    plt.show()
else:
    logger.info("No missing values in Yahoo Finance data.")

In [ ]:
# Cell 13: Summary statistics
print("Summary statistics (full period):")
display(nvda_yf.describe())

# Split into pre-2020 and post-2020 to see changes
pre_2020 = nvda_yf[nvda_yf.index < '2020-01-01']
post_2020 = nvda_yf[nvda_yf.index >= '2020-01-01']

print("\nPre-2020 statistics:")
display(pre_2020.describe())
print("\nPost-2020 statistics:")
display(post_2020.describe())

In [ ]:
# Cell 14: Plot closing price with volume
fig, axes = plt.subplots(2, 1, figsize=(16, 10), sharex=True)

# Price plot
axes[0].plot(nvda_yf.index, nvda_yf['Close'], label='Close Price', color='blue')
axes[0].set_title(f'{TICKER} Closing Price Over Time', fontsize=16)
axes[0].set_ylabel('Price ($)')
axes[0].legend()
axes[0].grid(True)

# Volume plot
axes[1].bar(nvda_yf.index, nvda_yf['Volume'], color='gray', alpha=0.6, width=2)
axes[1].set_title('Trading Volume', fontsize=16)
axes[1].set_ylabel('Volume')
axes[1].set_xlabel('Date')
axes[1].grid(True)

plt.tight_layout()
plt.savefig(os.path.join(PROCESSED_DATA_PATH, "price_volume.png"))
plt.show()

In [ ]:
# Cell 15: Calculate daily returns and visualize
nvda_yf['Return'] = nvda_yf['Close'].pct_change() * 100  # percentage returns
nvda_yf['Log_Return'] = np.log(nvda_yf['Close'] / nvda_yf['Close'].shift(1))

fig, axes = plt.subplots(3, 1, figsize=(16, 12))

# Returns over time
axes[0].plot(nvda_yf.index, nvda_yf['Return'], color='green', alpha=0.7, linewidth=0.8)
axes[0].axhline(y=0, color='black', linestyle='--', linewidth=0.8)
axes[0].set_title('Daily Percentage Returns', fontsize=14)
axes[0].set_ylabel('Return (%)')
axes[0].grid(True)

# Histogram of returns
axes[1].hist(nvda_yf['Return'].dropna(), bins=100, color='skyblue', edgecolor='black', alpha=0.7)
axes[1].set_title('Distribution of Daily Returns', fontsize=14)
axes[1].set_xlabel('Return (%)')
axes[1].set_ylabel('Frequency')
axes[1].axvline(x=0, color='red', linestyle='--')
axes[1].grid(True)

# Q-Q plot for normality
from scipy import stats
stats.probplot(nvda_yf['Return'].dropna(), dist="norm", plot=axes[2])
axes[2].set_title('Q-Q Plot of Returns', fontsize=14)
axes[2].grid(True)

plt.tight_layout()
plt.savefig(os.path.join(PROCESSED_DATA_PATH, "returns_analysis.png"))
plt.show()

# Summary stats for returns
print("Return statistics:")
print(nvda_yf['Return'].describe())

In [ ]:
# Cell 16: Check stationarity of closing price and returns
def check_stationarity(series, title):
    result = adfuller(series.dropna())
    print(f'ADF Statistic for {title}: {result[0]:.6f}')
    print(f'p-value: {result[1]:.6f}')
    print('Critical values:')
    for key, value in result[4].items():
        print(f'\t{key}: {value:.3f}')
    if result[1] <= 0.05:
        print("=> Series is stationary (reject H0)\n")
    else:
        print("=> Series is non-stationary (fail to reject H0)\n")

check_stationarity(nvda_yf['Close'], 'Close Price')
check_stationarity(nvda_yf['Return'], 'Returns')

In [ ]:
# Cell 17: Autocorrelation and Partial Autocorrelation
fig, axes = plt.subplots(2, 2, figsize=(16, 8))

# ACF and PACF of closing price (non-stationary, just for illustration)
plot_acf(nvda_yf['Close'].dropna(), lags=40, ax=axes[0,0])
axes[0,0].set_title('ACF of Close Price')
plot_pacf(nvda_yf['Close'].dropna(), lags=40, ax=axes[0,1])
axes[0,1].set_title('PACF of Close Price')

# ACF and PACF of returns (stationary)
plot_acf(nvda_yf['Return'].dropna(), lags=40, ax=axes[1,0])
axes[1,0].set_title('ACF of Returns')
plot_pacf(nvda_yf['Return'].dropna(), lags=40, ax=axes[1,1])
axes[1,1].set_title('PACF of Returns')

plt.tight_layout()
plt.savefig(os.path.join(PROCESSED_DATA_PATH, "acf_pacf.png"))
plt.show()

In [ ]:
# Cell 18: Technical Indicators using 'ta' library
import ta

# Add common indicators
nvda_yf['SMA_10'] = ta.trend.sma_indicator(nvda_yf['Close'], window=10)
nvda_yf['SMA_30'] = ta.trend.sma_indicator(nvda_yf['Close'], window=30)
nvda_yf['EMA_12'] = ta.trend.ema_indicator(nvda_yf['Close'], window=12)
nvda_yf['EMA_26'] = ta.trend.ema_indicator(nvda_yf['Close'], window=26)
nvda_yf['RSI'] = ta.momentum.rsi(nvda_yf['Close'], window=14)
macd = ta.trend.MACD(nvda_yf['Close'])
nvda_yf['MACD'] = macd.macd()
nvda_yf['MACD_signal'] = macd.macd_signal()
nvda_yf['MACD_diff'] = macd.macd_diff()

# Bollinger Bands
bb = ta.volatility.BollingerBands(nvda_yf['Close'], window=20, window_dev=2)
nvda_yf['BB_high'] = bb.bollinger_hband()
nvda_yf['BB_low'] = bb.bollinger_lband()
nvda_yf['BB_width'] = bb.bollinger_wband()

# Volume-based: On-Balance Volume (OBV)
nvda_yf['OBV'] = ta.volume.on_balance_volume(nvda_yf['Close'], nvda_yf['Volume'])

logger.info("Technical indicators added.")

In [ ]:
# Cell 19: Visualize some indicators
fig, axes = plt.subplots(4, 1, figsize=(16, 14), sharex=True)

# Price with SMAs
axes[0].plot(nvda_yf.index, nvda_yf['Close'], label='Close', alpha=0.6, linewidth=1)
axes[0].plot(nvda_yf.index, nvda_yf['SMA_10'], label='SMA 10', linestyle='--')
axes[0].plot(nvda_yf.index, nvda_yf['SMA_30'], label='SMA 30', linestyle='--')
axes[0].set_title('Price with Moving Averages')
axes[0].legend()
axes[0].grid(True)

# RSI
axes[1].plot(nvda_yf.index, nvda_yf['RSI'], color='purple')
axes[1].axhline(y=70, color='red', linestyle='--', label='Overbought (70)')
axes[1].axhline(y=30, color='green', linestyle='--', label='Oversold (30)')
axes[1].set_title('Relative Strength Index (RSI)')
axes[1].set_ylim(0, 100)
axes[1].legend()
axes[1].grid(True)

# MACD
axes[2].plot(nvda_yf.index, nvda_yf['MACD'], label='MACD', color='blue')
axes[2].plot(nvda_yf.index, nvda_yf['MACD_signal'], label='Signal', color='orange')
axes[2].bar(nvda_yf.index, nvda_yf['MACD_diff'], label='Histogram', color='gray', alpha=0.5)
axes[2].set_title('MACD')
axes[2].legend()
axes[2].grid(True)

# Bollinger Band Width (volatility)
axes[3].fill_between(nvda_yf.index, nvda_yf['BB_low'], nvda_yf['BB_high'], alpha=0.3, color='gray', label='Bollinger Bands')
axes[3].plot(nvda_yf.index, nvda_yf['Close'], label='Close', color='black', linewidth=1)
axes[3].set_title('Price with Bollinger Bands')
axes[3].legend()
axes[3].grid(True)

plt.tight_layout()
plt.savefig(os.path.join(PROCESSED_DATA_PATH, "technical_indicators.png"))
plt.show()

In [ ]:
# Cell 20: Merge VIX and check correlation
nvda_yf = nvda_yf.join(vix, how='left')
# Forward fill VIX (since VIX is not available on all days? Actually VIX has trading days, so align)
nvda_yf['VIX'] = nvda_yf['VIX'].fillna(method='ffill')

# Compute correlation matrix for selected features
features_for_corr = ['Close', 'Volume', 'Return', 'RSI', 'MACD', 'VIX', 'OBV', 'BB_width']
corr_matrix = nvda_yf[features_for_corr].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f', square=True)
plt.title('Correlation Matrix of Selected Features')
plt.tight_layout()
plt.savefig(os.path.join(PROCESSED_DATA_PATH, "correlation_matrix.png"))
plt.show()

In [ ]:
# Cell 21: Seasonality analysis – average returns by day of week, month
nvda_yf['DayOfWeek'] = nvda_yf.index.dayofweek  # Monday=0, Sunday=6
nvda_yf['Month'] = nvda_yf.index.month
nvda_yf['Year'] = nvda_yf.index.year

# Average return by day of week
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
day_avg = nvda_yf.groupby('DayOfWeek')['Return'].mean().rename(dict(enumerate(day_order)))

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
day_avg.plot(kind='bar', ax=axes[0], color='skyblue')
axes[0].set_title('Average Daily Return by Weekday')
axes[0].set_ylabel('Avg Return (%)')
axes[0].set_xticklabels(day_order, rotation=45)

# Average return by month
month_avg = nvda_yf.groupby('Month')['Return'].mean()
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
month_avg.index = month_names
month_avg.plot(kind='bar', ax=axes[1], color='lightgreen')
axes[1].set_title('Average Daily Return by Month')
axes[1].set_ylabel('Avg Return (%)')
axes[1].set_xticklabels(month_names, rotation=45)

plt.tight_layout()
plt.savefig(os.path.join(PROCESSED_DATA_PATH, "seasonality.png"))
plt.show()

In [ ]:
# Cell 22: Check for outliers in returns
# Using IQR method
Q1 = nvda_yf['Return'].quantile(0.25)
Q3 = nvda_yf['Return'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers = nvda_yf[(nvda_yf['Return'] < lower_bound) | (nvda_yf['Return'] > upper_bound)]
print(f"Number of outlier days (returns): {len(outliers)} ({len(outliers)/len(nvda_yf)*100:.2f}%)")
print("Outlier dates and returns:")
print(outliers[['Return']].sort_values('Return'))

# Plot with outliers highlighted
plt.figure(figsize=(16, 4))
plt.plot(nvda_yf.index, nvda_yf['Return'], color='blue', alpha=0.5)
plt.scatter(outliers.index, outliers['Return'], color='red', s=20, label='Outliers')
plt.axhline(y=0, color='black', linestyle='--')
plt.title('Daily Returns with Outliers Highlighted')
plt.ylabel('Return (%)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(PROCESSED_DATA_PATH, "outliers.png"))
plt.show()

In [ ]:
# Cell 23: Quick look at high-frequency MTBench data (if available)
if not nvda_mtbench.empty:
    logger.info("Exploring MTBench high-frequency data for NVDA...")
    # Convert timestamp column to datetime
    nvda_mtbench['timestamp'] = pd.to_datetime(nvda_mtbench['timestamp'])
    nvda_mtbench.set_index('timestamp', inplace=True)
    
    # Plot one day of 5-min data as example
    sample_day = '2023-06-01'
    if sample_day in nvda_mtbench.index:
        day_data = nvda_mtbench.loc[sample_day]
        plt.figure(figsize=(14, 5))
        plt.plot(day_data.index, day_data['close'], marker='o', linestyle='-', markersize=3)
        plt.title(f'NVDA 5-minute Price on {sample_day}')
        plt.xlabel('Time')
        plt.ylabel('Price ($)')
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(os.path.join(PROCESSED_DATA_PATH, "mtbench_sample.png"))
        plt.show()
    else:
        logger.info(f"Sample day {sample_day} not in MTBench data.")

In [ ]:
# Cell 24: Save processed daily data with features for modeling
# Drop rows with NaN from indicator calculation (first few rows)
nvda_processed = nvda_yf.dropna().copy()
nvda_processed.to_csv(os.path.join(PROCESSED_DATA_PATH, f"{TICKER}_processed.csv"))
logger.info(f"Processed daily data saved: {nvda_processed.shape[0]} rows, {nvda_processed.shape[1]} columns")

# Also save the feature list
with open(os.path.join(PROCESSED_DATA_PATH, "feature_list.txt"), "w") as f:
    f.write("\n".join(nvda_processed.columns.tolist()))

logger.info("Phase 2 complete. EDA results and processed data saved.")

## Feature Engineering & Preprocessing

In [ ]:
# Cell 26: Load the processed daily data
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
import joblib

PROCESSED_DATA_PATH = "data/processed"
TICKER = "NVDA"

# Load data
df = pd.read_csv(os.path.join(PROCESSED_DATA_PATH, f"{TICKER}_processed.csv"), index_col=0, parse_dates=True)
print(f"Data shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"Date range: {df.index.min()} to {df.index.max()}")

In [ ]:
# Cell 27: Define target variables
# For regression: next day's closing price
df['Target_Close'] = df['Close'].shift(-1)

# For classification: next day's direction (1 if up, 0 if down)
df['Target_Dir'] = (df['Target_Close'] > df['Close']).astype(int)

# Drop rows with NaN targets (last row)
df.dropna(subset=['Target_Close', 'Target_Dir'], inplace=True)

print(f"After target creation: {df.shape}")

In [ ]:
# Cell 28: Feature engineering – additional predictors
# Rolling volatility (standard deviation of returns over 20 days)
df['Volatility_20'] = df['Return'].rolling(20).std()

# Price momentum: (Close / Close shifted n) - 1
for lag in [5, 10, 21]:
    df[f'Momentum_{lag}'] = df['Close'] / df['Close'].shift(lag) - 1

# Volume change
df['Volume_change'] = df['Volume'].pct_change()

# RSI change
df['RSI_change'] = df['RSI'].diff()

# MACD histogram change
df['MACD_hist_change'] = df['MACD_diff'].diff()

# Day of week (already created, but ensure it's cyclical encoded)
df['Day_sin'] = np.sin(2 * np.pi * df['DayOfWeek'] / 7)
df['Day_cos'] = np.cos(2 * np.pi * df['DayOfWeek'] / 7)

# Month cyclical encoding
df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)

# Drop original categorical columns (optional)
df.drop(['DayOfWeek', 'Month', 'Year'], axis=1, inplace=True, errors='ignore')

# Drop rows with NaN from new features
df.dropna(inplace=True)
print(f"After feature engineering: {df.shape}")

In [ ]:
# Cell 29: Select features for modeling
# We'll choose a set of features that are likely useful.
# Exclude price columns that would cause look-ahead if used carelessly.
feature_columns = [
    'Open', 'High', 'Low', 'Close', 'Volume',
    'SMA_10', 'SMA_30', 'EMA_12', 'EMA_26',
    'RSI', 'MACD', 'MACD_signal', 'MACD_diff',
    'BB_high', 'BB_low', 'BB_width', 'OBV',
    'VIX', 'Return', 'Log_Return',
    'Volatility_20', 'Momentum_5', 'Momentum_10', 'Momentum_21',
    'Volume_change', 'RSI_change', 'MACD_hist_change',
    'Day_sin', 'Day_cos', 'Month_sin', 'Month_cos'
]

# Ensure all selected columns exist in df
feature_columns = [col for col in feature_columns if col in df.columns]
print(f"Number of features: {len(feature_columns)}")
print("Features:", feature_columns)

# Separate features and targets
X = df[feature_columns]
y_reg = df['Target_Close']
y_clf = df['Target_Dir']

In [ ]:
# Cell 30: Chronological split
# Define split dates based on typical train/val/test proportions (e.g., 70/15/15)
total_len = len(df)
train_end = int(0.7 * total_len)
val_end = int(0.85 * total_len)

# Get the corresponding dates
train_dates = df.index[:train_end]
val_dates = df.index[train_end:val_end]
test_dates = df.index[val_end:]

print(f"Train: {train_dates[0]} to {train_dates[-1]} ({len(train_dates)} days)")
print(f"Validation: {val_dates[0]} to {val_dates[-1]} ({len(val_dates)} days)")
print(f"Test: {test_dates[0]} to {test_dates[-1]} ({len(test_dates)} days)")

# Split indices
X_train = X.loc[train_dates]
y_reg_train = y_reg.loc[train_dates]
y_clf_train = y_clf.loc[train_dates]

X_val = X.loc[val_dates]
y_reg_val = y_reg.loc[val_dates]
y_clf_val = y_clf.loc[val_dates]

X_test = X.loc[test_dates]
y_reg_test = y_reg.loc[test_dates]
y_clf_test = y_clf.loc[test_dates]

print(f"\nShapes:")
print(f"X_train: {X_train.shape}, y_reg_train: {y_reg_train.shape}")
print(f"X_val: {X_val.shape}, y_val: {y_reg_val.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_reg_test.shape}")

In [ ]:
# Cell 31: Scale the features
# Fit scaler on training data only, then transform all sets
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Save the scaler for later use (e.g., inverse transform predictions)
scaler_path = os.path.join(PROCESSED_DATA_PATH, "scaler.pkl")
joblib.dump(scaler, scaler_path)
print(f"Scaler saved to {scaler_path}")

# Convert back to DataFrames with column names (optional)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_columns, index=X_train.index)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=feature_columns, index=X_val.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_columns, index=X_test.index)

In [ ]:
# Cell 32: Create sequences for RNN/LSTM/GRU/Transformer models
def create_sequences(X, y, window_size):
    X_seq, y_seq = [], []
    for i in range(window_size, len(X)):
        X_seq.append(X[i-window_size:i])
        y_seq.append(y[i])
    return np.array(X_seq), np.array(y_seq)

WINDOW_SIZE = 60  # use 60 days of history to predict next day

# Create sequences for each set
X_train_seq, y_train_seq = create_sequences(X_train_scaled.values, y_reg_train.values, WINDOW_SIZE)
X_val_seq, y_val_seq = create_sequences(X_val_scaled.values, y_reg_val.values, WINDOW_SIZE)
X_test_seq, y_test_seq = create_sequences(X_test_scaled.values, y_reg_test.values, WINDOW_SIZE)

print(f"Training sequences: {X_train_seq.shape}, targets: {y_train_seq.shape}")
print(f"Validation sequences: {X_val_seq.shape}, targets: {y_val_seq.shape}")
print(f"Test sequences: {X_test_seq.shape}, targets: {y_test_seq.shape}")

# For classification, we could also create sequences using y_clf.
# We'll handle that separately when building classification models.

In [ ]:
# Cell 33: Save the prepared sequences for modeling
np.save(os.path.join(PROCESSED_DATA_PATH, "X_train_seq.npy"), X_train_seq)
np.save(os.path.join(PROCESSED_DATA_PATH, "y_train_seq.npy"), y_train_seq)
np.save(os.path.join(PROCESSED_DATA_PATH, "X_val_seq.npy"), X_val_seq)
np.save(os.path.join(PROCESSED_DATA_PATH, "y_val_seq.npy"), y_val_seq)
np.save(os.path.join(PROCESSED_DATA_PATH, "X_test_seq.npy"), X_test_seq)
np.save(os.path.join(PROCESSED_DATA_PATH, "y_test_seq.npy"), y_test_seq)

# Also save the dates corresponding to each sequence element for reference
# The target date for each sequence is the date of y (i.e., the prediction day)
train_seq_dates = X_train.index[WINDOW_SIZE:]
val_seq_dates = X_val.index[WINDOW_SIZE:]
test_seq_dates = X_test.index[WINDOW_SIZE:]

np.save(os.path.join(PROCESSED_DATA_PATH, "train_seq_dates.npy"), train_seq_dates)
np.save(os.path.join(PROCESSED_DATA_PATH, "val_seq_dates.npy"), val_seq_dates)
np.save(os.path.join(PROCESSED_DATA_PATH, "test_seq_dates.npy"), test_seq_dates)

print("Sequence data saved.")

In [ ]:
# Cell 34: Quick sanity check – plot a few sequences
plt.figure(figsize=(12, 4))
for i in range(3):
    plt.subplot(1, 3, i+1)
    # Plot the first feature (Close price) from a few sequences
    idx = i * 100  # spacing
    if idx < len(X_train_seq):
        plt.plot(X_train_seq[idx, :, feature_columns.index('Close')])
        plt.title(f'Train Seq {idx} - Close')
plt.tight_layout()
plt.savefig(os.path.join(PROCESSED_DATA_PATH, "sequence_samples.png"))
plt.show()

In [ ]:
# Cell 35: (Optional) Handle high-frequency MTBench data for future use
# If MTBench data was successfully loaded and processed, we might want to downsample it to daily
# or use it directly in a hybrid model. This is a placeholder for future expansion.
if os.path.exists(os.path.join(RAW_DATA_PATH, "NVDA_MTBench.csv")):
    print("MTBench data available. Consider integrating it later.")
    # For now, we'll just note it.
else:
    print("MTBench data not found – continuing with daily data only.")

logger.info("Phase 3 complete. Data ready for modeling.")

## Baseline ARIMA Model

In [ ]:
# Cell 36: Import additional libraries for ARIMA
import pmdarima as pm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib
import warnings
warnings.filterwarnings('ignore')

logger.info("Starting Phase 4: ARIMA Baseline Model")

In [ ]:
# Cell 37: Load the processed data (if starting fresh)
import pandas as pd
import numpy as np
import os

PROCESSED_DATA_PATH = "data/processed"
TICKER = "NVDA"

# Load the daily data with targets
df = pd.read_csv(os.path.join(PROCESSED_DATA_PATH, f"{TICKER}_processed.csv"), index_col=0, parse_dates=True)
print(f"Data shape: {df.shape}")
print(f"Date range: {df.index.min()} to {df.index.max()}")

# Extract the closing price series
close_series = df['Close'].copy()

In [ ]:
# Cell 38: Use the same chronological split as Phase 3
# We need the split indices for walk-forward validation
# Load the saved split dates (or recompute from indices)
train_dates = df.index[:int(0.7*len(df))]
val_dates = df.index[int(0.7*len(df)):int(0.85*len(df))]
test_dates = df.index[int(0.85*len(df)):]

# Split the series accordingly
train_close = close_series.loc[train_dates]
val_close = close_series.loc[val_dates]
test_close = close_series.loc[test_dates]

print(f"Train: {train_close.index[0]} to {train_close.index[-1]} ({len(train_close)} days)")
print(f"Validation: {val_close.index[0]} to {val_close.index[-1]} ({len(val_close)} days)")
print(f"Test: {test_close.index[0]} to {test_close.index[-1]} ({len(test_close)} days)")

In [ ]:
# Cell 39: Check stationarity again (for completeness)
from statsmodels.tsa.stattools import adfuller

def adf_test(series, title):
    result = adfuller(series.dropna())
    print(f'{title} ADF Statistic: {result[0]:.6f}')
    print(f'p-value: {result[1]:.6f}')
    if result[1] <= 0.05:
        print("=> Series is stationary\n")
    else:
        print("=> Series is non-stationary\n")

adf_test(train_close, 'Train Close Price')
adf_test(train_close.diff().dropna(), 'Train Close Price (1st diff)')

In [ ]:
# Cell 40: Automatic ARIMA order selection using auto_arima
# We'll use pmdarima which handles seasonal patterns automatically (though we don't expect strong seasonality).
# Set a maximum order to avoid excessive computation time.

logger.info("Running auto_arima to find best (p,d,q) on training data...")
auto_model = pm.auto_arima(
    train_close,
    start_p=0, max_p=10,
    start_q=0, max_q=10,
    start_P=0, max_P=2,
    start_Q=0, max_Q=2,
    seasonal=False,           # set to True if you suspect seasonality; we'll keep False for daily data
    test='adf',                # use ADF test to determine d
    trace=True,
    error_action='ignore',
    suppress_warnings=True,
    stepwise=True,
    n_fits=50,
    information_criterion='aic',
    random_state=42
)

print("\nBest ARIMA model:", auto_model.order)
print("AIC:", auto_model.aic())
print(auto_model.summary())

# Save the auto_arima model (it contains the fitted ARIMA)
model_path = os.path.join("models", "auto_arima_model.pkl")
joblib.dump(auto_model, model_path)
logger.info(f"Auto ARIMA model saved to {model_path}")

In [ ]:
# Cell 41: Walk-forward validation on validation set
# We'll retrain the ARIMA model with the chosen order on an expanding window.
# This simulates how the model would be used in practice.

best_order = auto_model.order  # (p,d,q) from auto_arima

def walk_forward_arima(train_series, test_series, order):
    """
    Perform walk-forward validation for ARIMA.
    For each day in test_series, refit the model on all available data up to that day,
    then forecast one step ahead.
    """
    history = list(train_series)
    predictions = []
    for t in range(len(test_series)):
        model = ARIMA(history, order=order)
        model_fit = model.fit()
        forecast = model_fit.forecast(steps=1)[0]
        predictions.append(forecast)
        # Append the actual observation to history for next step
        history.append(test_series.iloc[t])
    return np.array(predictions)

# First on validation set
logger.info("Running walk-forward validation on validation set...")
val_pred = walk_forward_arima(train_close, val_close, best_order)

# Evaluate
val_actual = val_close.values
val_rmse = np.sqrt(mean_squared_error(val_actual, val_pred))
val_mae = mean_absolute_error(val_actual, val_pred)
val_mape = np.mean(np.abs((val_actual - val_pred) / val_actual)) * 100

# Directional accuracy
val_direction_actual = (val_actual[1:] > val_actual[:-1]).astype(int)
val_direction_pred = (val_pred[1:] > val_actual[:-1]).astype(int)  # compare predicted next to current actual
val_dir_acc = np.mean(val_direction_pred == val_direction_actual) * 100

print("\nValidation Set Performance:")
print(f"RMSE: {val_rmse:.4f}")
print(f"MAE: {val_mae:.4f}")
print(f"MAPE: {val_mape:.2f}%")
print(f"Directional Accuracy: {val_dir_acc:.2f}%")

In [ ]:
# Cell 42: Evaluate on test set using walk-forward validation (starting from combined train+val)
logger.info("Running walk-forward validation on test set...")
# Combine train and validation for history before test
combined_train_val = pd.concat([train_close, val_close])
test_pred = walk_forward_arima(combined_train_val, test_close, best_order)

test_actual = test_close.values
test_rmse = np.sqrt(mean_squared_error(test_actual, test_pred))
test_mae = mean_absolute_error(test_actual, test_pred)
test_mape = np.mean(np.abs((test_actual - test_pred) / test_actual)) * 100

test_direction_actual = (test_actual[1:] > test_actual[:-1]).astype(int)
test_direction_pred = (test_pred[1:] > test_actual[:-1]).astype(int)
test_dir_acc = np.mean(test_direction_pred == test_direction_actual) * 100

print("\nTest Set Performance:")
print(f"RMSE: {test_rmse:.4f}")
print(f"MAE: {test_mae:.4f}")
print(f"MAPE: {test_mape:.2f}%")
print(f"Directional Accuracy: {test_dir_acc:.2f}%")

In [ ]:
# Cell 43: Plot validation and test predictions vs actual
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Validation
axes[0].plot(val_close.index, val_actual, label='Actual', color='blue')
axes[0].plot(val_close.index, val_pred, label='ARIMA Predicted', color='red', linestyle='--')
axes[0].set_title(f'ARIMA Walk-Forward Validation – Validation Set (Order {best_order})')
axes[0].set_ylabel('Price ($)')
axes[0].legend()
axes[0].grid(True)

# Test
axes[1].plot(test_close.index, test_actual, label='Actual', color='blue')
axes[1].plot(test_close.index, test_pred, label='ARIMA Predicted', color='red', linestyle='--')
axes[1].set_title(f'ARIMA Walk-Forward Validation – Test Set')
axes[1].set_ylabel('Price ($)')
axes[1].set_xlabel('Date')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig(os.path.join(PROCESSED_DATA_PATH, "arima_predictions.png"))
plt.show()

In [ ]:
# Cell 44: Save ARIMA predictions for later comparison
results_df = pd.DataFrame({
    'date': test_close.index,
    'actual': test_actual,
    'arima_pred': test_pred,
    'direction_actual': np.concatenate([test_direction_actual, [np.nan]]),  # align lengths
    'direction_pred': np.concatenate([test_direction_pred, [np.nan]])
})
results_df.to_csv(os.path.join(PROCESSED_DATA_PATH, "arima_test_results.csv"), index=False)
logger.info("ARIMA test results saved.")

In [ ]:
# Cell 45: Diagnostics – Residual analysis
# Fit ARIMA on full training+validation and check residuals
final_model = ARIMA(combined_train_val, order=best_order)
final_fit = final_model.fit()
residuals = final_fit.resid

fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Residuals over time
axes[0,0].plot(residuals)
axes[0,0].set_title('Residuals')
axes[0,0].axhline(y=0, color='red', linestyle='--')

# Histogram of residuals
axes[0,1].hist(residuals, bins=30, edgecolor='black')
axes[0,1].set_title('Residuals Histogram')

# ACF of residuals
plot_acf(residuals, lags=40, ax=axes[1,0])
axes[1,0].set_title('ACF of Residuals')

# PACF of residuals
plot_pacf(residuals, lags=40, ax=axes[1,1])
axes[1,1].set_title('PACF of Residuals')

plt.tight_layout()
plt.savefig(os.path.join(PROCESSED_DATA_PATH, "arima_residuals.png"))
plt.show()

# Ljung-Box test for autocorrelation in residuals
from statsmodels.stats.diagnostic import acorr_ljungbox
lb_test = acorr_ljungbox(residuals, lags=[10,20,30], return_df=True)
print("Ljung-Box test p-values:")
print(lb_test)

In [ ]:
# Cell 46: Summary of ARIMA baseline
print("\n" + "="*60)
print("ARIMA BASELINE SUMMARY")
print("="*60)
print(f"Best order (p,d,q): {best_order}")
print(f"AIC on training: {auto_model.aic():.2f}")
print("\nValidation Set:")
print(f"  RMSE: {val_rmse:.4f}   MAE: {val_mae:.4f}   MAPE: {val_mape:.2f}%   DirAcc: {val_dir_acc:.2f}%")
print("\nTest Set:")
print(f"  RMSE: {test_rmse:.4f}   MAE: {test_mae:.4f}   MAPE: {test_mape:.2f}%   DirAcc: {test_dir_acc:.2f}%")
print("="*60)

logger.info("Phase 4 complete. ARIMA baseline established.")

## LSTM Model

In [ ]:
# Cell 47: Import additional libraries for deep learning
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
import joblib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os

logger.info("Starting Phase 5: LSTM Model")

In [ ]:
# Cell 48: Load preprocessed sequences and targets
PROCESSED_DATA_PATH = "data/processed"

X_train_seq = np.load(os.path.join(PROCESSED_DATA_PATH, "X_train_seq.npy"))
y_train_seq = np.load(os.path.join(PROCESSED_DATA_PATH, "y_train_seq.npy"))
X_val_seq = np.load(os.path.join(PROCESSED_DATA_PATH, "X_val_seq.npy"))
y_val_seq = np.load(os.path.join(PROCESSED_DATA_PATH, "y_val_seq.npy"))
X_test_seq = np.load(os.path.join(PROCESSED_DATA_PATH, "X_test_seq.npy"))
y_test_seq = np.load(os.path.join(PROCESSED_DATA_PATH, "y_test_seq.npy"))

# Load dates if needed (optional)
train_seq_dates = np.load(os.path.join(PROCESSED_DATA_PATH, "train_seq_dates.npy"), allow_pickle=True)
val_seq_dates = np.load(os.path.join(PROCESSED_DATA_PATH, "val_seq_dates.npy"), allow_pickle=True)
test_seq_dates = np.load(os.path.join(PROCESSED_DATA_PATH, "test_seq_dates.npy"), allow_pickle=True)

print(f"X_train_seq shape: {X_train_seq.shape}")
print(f"y_train_seq shape: {y_train_seq.shape}")
print(f"X_val_seq shape: {X_val_seq.shape}")
print(f"X_test_seq shape: {X_test_seq.shape}")

In [ ]:
# Cell 49: Scale the target values for better training stability
# We'll fit a scaler on the training targets only.
target_scaler = MinMaxScaler(feature_range=(0, 1))

# Reshape to 2D for scaler
y_train_scaled = target_scaler.fit_transform(y_train_seq.reshape(-1, 1)).flatten()
y_val_scaled = target_scaler.transform(y_val_seq.reshape(-1, 1)).flatten()
y_test_scaled = target_scaler.transform(y_test_seq.reshape(-1, 1)).flatten()

# Save target scaler for inverse transformation later
scaler_path = os.path.join(PROCESSED_DATA_PATH, "target_scaler.pkl")
joblib.dump(target_scaler, scaler_path)
print(f"Target scaler saved to {scaler_path}")

In [ ]:
# Cell 50: Define LSTM model architecture
def build_lstm_model(input_shape, lstm_units=[50, 50], dropout_rate=0.2, learning_rate=0.001):
    model = Sequential()
    model.add(Input(shape=input_shape))
    
    # First LSTM layer with return sequences for stacking
    model.add(LSTM(units=lstm_units[0], return_sequences=True))
    model.add(Dropout(dropout_rate))
    
    # Second LSTM layer
    model.add(LSTM(units=lstm_units[1], return_sequences=False))
    model.add(Dropout(dropout_rate))
    
    # Dense layers
    model.add(Dense(25, activation='relu'))
    model.add(Dropout(dropout_rate/2))
    model.add(Dense(1))  # Linear activation for regression
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

# Model parameters
input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])  # (window_size, n_features)
lstm_units = [64, 32]  # can be tuned
dropout_rate = 0.3
learning_rate = 0.001

model = build_lstm_model(input_shape, lstm_units, dropout_rate, learning_rate)
model.summary()

In [ ]:
# Cell 51: Set up callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1),
    ModelCheckpoint(
        filepath=os.path.join("models", "lstm_best.keras"),
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)
]

logger.info("Callbacks configured.")

In [ ]:
# Cell 52: Train the model
history = model.fit(
    X_train_seq, y_train_scaled,
    validation_data=(X_val_seq, y_val_scaled),
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

# Save training history
import pickle
with open(os.path.join("models", "lstm_history.pkl"), 'wb') as f:
    pickle.dump(history.history, f)

logger.info("LSTM training completed.")

In [ ]:
# Cell 53: Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Loss
axes[0].plot(history.history['loss'], label='Train Loss')
axes[0].plot(history.history['val_loss'], label='Val Loss')
axes[0].set_title('Model Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss (MSE)')
axes[0].legend()
axes[0].grid(True)

# MAE
axes[1].plot(history.history['mae'], label='Train MAE')
axes[1].plot(history.history['val_mae'], label='Val MAE')
axes[1].set_title('Model MAE')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MAE')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig(os.path.join(PROCESSED_DATA_PATH, "lstm_training_history.png"))
plt.show()

In [ ]:
# Cell 54: Load the best model (saved by ModelCheckpoint)
best_model = load_model(os.path.join("models", "lstm_best.keras"))
logger.info("Best LSTM model loaded.")

In [ ]:
# Cell 55: Evaluate on validation and test sets
# Validation set predictions
y_val_pred_scaled = best_model.predict(X_val_seq, verbose=0)
y_val_pred = target_scaler.inverse_transform(y_val_pred_scaled).flatten()
y_val_actual = y_val_seq  # already original scale

# Test set predictions
y_test_pred_scaled = best_model.predict(X_test_seq, verbose=0)
y_test_pred = target_scaler.inverse_transform(y_test_pred_scaled).flatten()
y_test_actual = y_test_seq

# Compute metrics
def compute_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    mape = np.mean(np.abs((actual - pred) / actual)) * 100
    # Directional accuracy
    direction_actual = (actual[1:] > actual[:-1]).astype(int)
    direction_pred = (pred[1:] > actual[:-1]).astype(int)
    dir_acc = np.mean(direction_pred == direction_actual) * 100
    return rmse, mae, mape, dir_acc

val_rmse, val_mae, val_mape, val_dir_acc = compute_metrics(y_val_actual, y_val_pred)
test_rmse, test_mae, test_mape, test_dir_acc = compute_metrics(y_test_actual, y_test_pred)

print("\nLSTM Validation Set Performance:")
print(f"RMSE: {val_rmse:.4f}   MAE: {val_mae:.4f}   MAPE: {val_mape:.2f}%   DirAcc: {val_dir_acc:.2f}%")

print("\nLSTM Test Set Performance:")
print(f"RMSE: {test_rmse:.4f}   MAE: {test_mae:.4f}   MAPE: {test_mape:.2f}%   DirAcc: {test_dir_acc:.2f}%")

In [ ]:
# Cell 56: Plot predictions vs actual
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Validation
axes[0].plot(val_seq_dates, y_val_actual, label='Actual', color='blue')
axes[0].plot(val_seq_dates, y_val_pred, label='LSTM Predicted', color='red', linestyle='--')
axes[0].set_title('LSTM Predictions – Validation Set')
axes[0].set_ylabel('Price ($)')
axes[0].legend()
axes[0].grid(True)

# Test
axes[1].plot(test_seq_dates, y_test_actual, label='Actual', color='blue')
axes[1].plot(test_seq_dates, y_test_pred, label='LSTM Predicted', color='red', linestyle='--')
axes[1].set_title('LSTM Predictions – Test Set')
axes[1].set_ylabel('Price ($)')
axes[1].set_xlabel('Date')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig(os.path.join(PROCESSED_DATA_PATH, "lstm_predictions.png"))
plt.show()

In [ ]:
# Cell 57: Compare with ARIMA baseline
# Load ARIMA test results
arima_results = pd.read_csv(os.path.join(PROCESSED_DATA_PATH, "arima_test_results.csv"))
# Note: ARIMA predictions are aligned with test set dates, but might have different length if walk-forward started earlier.
# We'll align by date if needed. For simplicity, we'll use the metrics we already computed.

print("\n" + "="*70)
print("MODEL COMPARISON ON TEST SET")
print("="*70)
print(f"{'Model':<15} {'RMSE':<10} {'MAE':<10} {'MAPE(%)':<10} {'DirAcc(%)':<10}")
print("-"*70)
print(f"{'ARIMA':<15} {test_rmse_arima:<10.4f} {test_mae_arima:<10.4f} {test_mape_arima:<10.2f} {test_dir_acc_arima:<10.2f}")
print(f"{'LSTM':<15} {test_rmse:<10.4f} {test_mae:<10.4f} {test_mape:<10.2f} {test_dir_acc:<10.2f}")
print("="*70)

# We need to retrieve ARIMA test metrics from earlier. Let's store them in a variable.
# For completeness, recompute from saved results if needed.
arima_test_rmse = np.sqrt(mean_squared_error(arima_results['actual'], arima_results['arima_pred']))
arima_test_mae = mean_absolute_error(arima_results['actual'], arima_results['arima_pred'])
arima_test_mape = np.mean(np.abs((arima_results['actual'] - arima_results['arima_pred']) / arima_results['actual'])) * 100
arima_dir_acc = arima_results['direction_pred'].iloc[:-1].eq(arima_results['direction_actual'].iloc[:-1]).mean() * 100

print("\nUpdated comparison with recomputed ARIMA metrics:")
print(f"ARIMA Test RMSE: {arima_test_rmse:.4f}, MAE: {arima_test_mae:.4f}, MAPE: {arima_test_mape:.2f}%, DirAcc: {arima_dir_acc:.2f}%")

In [ ]:
# Cell 58: Save LSTM predictions and metrics
lstm_results = pd.DataFrame({
    'date': test_seq_dates,
    'actual': y_test_actual,
    'lstm_pred': y_test_pred
})
lstm_results.to_csv(os.path.join(PROCESSED_DATA_PATH, "lstm_test_results.csv"), index=False)

# Save metrics
metrics_dict = {
    'model': 'LSTM',
    'val_rmse': val_rmse,
    'val_mae': val_mae,
    'val_mape': val_mape,
    'val_dir_acc': val_dir_acc,
    'test_rmse': test_rmse,
    'test_mae': test_mae,
    'test_mape': test_mape,
    'test_dir_acc': test_dir_acc
}
metrics_df = pd.DataFrame([metrics_dict])
metrics_df.to_csv(os.path.join(PROCESSED_DATA_PATH, "lstm_metrics.csv"), index=False)

logger.info("LSTM results saved.")

In [ ]:
# Cell 59: (Optional) Simple hyperparameter tuning
# We could run a quick grid search on a few key parameters, but given time we'll skip for now.
# If we wanted to, we'd use Keras Tuner or manual loops.
logger.info("Phase 5 complete. LSTM model trained and evaluated.")

## GRU Model

In [ ]:
# Cell 60: Import necessary libraries (if not already)
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import GRU, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import os
import pickle

logger.info("Starting Phase 6: GRU Model")

In [ ]:
# Cell 61: Load preprocessed sequences and targets
PROCESSED_DATA_PATH = "data/processed"

X_train_seq = np.load(os.path.join(PROCESSED_DATA_PATH, "X_train_seq.npy"))
y_train_seq = np.load(os.path.join(PROCESSED_DATA_PATH, "y_train_seq.npy"))
X_val_seq = np.load(os.path.join(PROCESSED_DATA_PATH, "X_val_seq.npy"))
y_val_seq = np.load(os.path.join(PROCESSED_DATA_PATH, "y_val_seq.npy"))
X_test_seq = np.load(os.path.join(PROCESSED_DATA_PATH, "X_test_seq.npy"))
y_test_seq = np.load(os.path.join(PROCESSED_DATA_PATH, "y_test_seq.npy"))

train_seq_dates = np.load(os.path.join(PROCESSED_DATA_PATH, "train_seq_dates.npy"), allow_pickle=True)
val_seq_dates = np.load(os.path.join(PROCESSED_DATA_PATH, "val_seq_dates.npy"), allow_pickle=True)
test_seq_dates = np.load(os.path.join(PROCESSED_DATA_PATH, "test_seq_dates.npy"), allow_pickle=True)

print(f"X_train_seq shape: {X_train_seq.shape}")
print(f"y_train_seq shape: {y_train_seq.shape}")
print(f"X_val_seq shape: {X_val_seq.shape}")
print(f"X_test_seq shape: {X_test_seq.shape}")

In [ ]:
# Cell 62: Load target scaler (fitted on training targets in Phase 5)
target_scaler = joblib.load(os.path.join(PROCESSED_DATA_PATH, "target_scaler.pkl"))

# Scale targets (use same scaler for consistency)
y_train_scaled = target_scaler.transform(y_train_seq.reshape(-1, 1)).flatten()
y_val_scaled = target_scaler.transform(y_val_seq.reshape(-1, 1)).flatten()
y_test_scaled = target_scaler.transform(y_test_seq.reshape(-1, 1)).flatten()

In [ ]:
# Cell 63: Define GRU model architecture
def build_gru_model(input_shape, gru_units=[64, 32], dropout_rate=0.3, learning_rate=0.001):
    model = Sequential()
    model.add(Input(shape=input_shape))
    
    # First GRU layer with return sequences for stacking
    model.add(GRU(units=gru_units[0], return_sequences=True))
    model.add(Dropout(dropout_rate))
    
    # Second GRU layer
    model.add(GRU(units=gru_units[1], return_sequences=False))
    model.add(Dropout(dropout_rate))
    
    # Dense layers
    model.add(Dense(25, activation='relu'))
    model.add(Dropout(dropout_rate/2))
    model.add(Dense(1))  # Linear activation for regression
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

# Model parameters (can be tuned)
input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])
gru_units = [64, 32]
dropout_rate = 0.3
learning_rate = 0.001

model = build_gru_model(input_shape, gru_units, dropout_rate, learning_rate)
model.summary()

In [ ]:
# Cell 64: Set up callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1),
    ModelCheckpoint(
        filepath=os.path.join("models", "gru_best.keras"),
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)
]

logger.info("Callbacks configured.")

In [ ]:
# Cell 65: Train the GRU model
history = model.fit(
    X_train_seq, y_train_scaled,
    validation_data=(X_val_seq, y_val_scaled),
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

# Save training history
with open(os.path.join("models", "gru_history.pkl"), 'wb') as f:
    pickle.dump(history.history, f)

logger.info("GRU training completed.")

In [ ]:
# Cell 66: Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Loss
axes[0].plot(history.history['loss'], label='Train Loss')
axes[0].plot(history.history['val_loss'], label='Val Loss')
axes[0].set_title('GRU Model Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss (MSE)')
axes[0].legend()
axes[0].grid(True)

# MAE
axes[1].plot(history.history['mae'], label='Train MAE')
axes[1].plot(history.history['val_mae'], label='Val MAE')
axes[1].set_title('GRU Model MAE')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MAE')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig(os.path.join(PROCESSED_DATA_PATH, "gru_training_history.png"))
plt.show()

In [ ]:
# Cell 67: Load the best GRU model
best_model = load_model(os.path.join("models", "gru_best.keras"))
logger.info("Best GRU model loaded.")

In [ ]:
# Cell 68: Evaluate on validation and test sets
# Validation predictions
y_val_pred_scaled = best_model.predict(X_val_seq, verbose=0)
y_val_pred = target_scaler.inverse_transform(y_val_pred_scaled).flatten()
y_val_actual = y_val_seq

# Test predictions
y_test_pred_scaled = best_model.predict(X_test_seq, verbose=0)
y_test_pred = target_scaler.inverse_transform(y_test_pred_scaled).flatten()
y_test_actual = y_test_seq

# Compute metrics
def compute_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    mape = np.mean(np.abs((actual - pred) / actual)) * 100
    direction_actual = (actual[1:] > actual[:-1]).astype(int)
    direction_pred = (pred[1:] > actual[:-1]).astype(int)
    dir_acc = np.mean(direction_pred == direction_actual) * 100
    return rmse, mae, mape, dir_acc

val_rmse, val_mae, val_mape, val_dir_acc = compute_metrics(y_val_actual, y_val_pred)
test_rmse, test_mae, test_mape, test_dir_acc = compute_metrics(y_test_actual, y_test_pred)

print("\nGRU Validation Set Performance:")
print(f"RMSE: {val_rmse:.4f}   MAE: {val_mae:.4f}   MAPE: {val_mape:.2f}%   DirAcc: {val_dir_acc:.2f}%")

print("\nGRU Test Set Performance:")
print(f"RMSE: {test_rmse:.4f}   MAE: {test_mae:.4f}   MAPE: {test_mape:.2f}%   DirAcc: {test_dir_acc:.2f}%")

In [ ]:
# Cell 69: Plot GRU predictions vs actual
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Validation
axes[0].plot(val_seq_dates, y_val_actual, label='Actual', color='blue')
axes[0].plot(val_seq_dates, y_val_pred, label='GRU Predicted', color='red', linestyle='--')
axes[0].set_title('GRU Predictions – Validation Set')
axes[0].set_ylabel('Price ($)')
axes[0].legend()
axes[0].grid(True)

# Test
axes[1].plot(test_seq_dates, y_test_actual, label='Actual', color='blue')
axes[1].plot(test_seq_dates, y_test_pred, label='GRU Predicted', color='red', linestyle='--')
axes[1].set_title('GRU Predictions – Test Set')
axes[1].set_ylabel('Price ($)')
axes[1].set_xlabel('Date')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig(os.path.join(PROCESSED_DATA_PATH, "gru_predictions.png"))
plt.show()

In [ ]:
# Cell 70: Compare with ARIMA and LSTM
# Load ARIMA test results
arima_results = pd.read_csv(os.path.join(PROCESSED_DATA_PATH, "arima_test_results.csv"))
arima_test_rmse = np.sqrt(mean_squared_error(arima_results['actual'], arima_results['arima_pred']))
arima_test_mae = mean_absolute_error(arima_results['actual'], arima_results['arima_pred'])
arima_test_mape = np.mean(np.abs((arima_results['actual'] - arima_results['arima_pred']) / arima_results['actual'])) * 100
arima_dir_acc = arima_results['direction_pred'].iloc[:-1].eq(arima_results['direction_actual'].iloc[:-1]).mean() * 100

# Load LSTM metrics
lstm_metrics = pd.read_csv(os.path.join(PROCESSED_DATA_PATH, "lstm_metrics.csv"))
lstm_test_rmse = lstm_metrics['test_rmse'].values[0]
lstm_test_mae = lstm_metrics['test_mae'].values[0]
lstm_test_mape = lstm_metrics['test_mape'].values[0]
lstm_test_dir_acc = lstm_metrics['test_dir_acc'].values[0]

print("\n" + "="*80)
print("MODEL COMPARISON ON TEST SET")
print("="*80)
print(f"{'Model':<15} {'RMSE':<12} {'MAE':<12} {'MAPE(%)':<12} {'DirAcc(%)':<12}")
print("-"*80)
print(f"{'ARIMA':<15} {arima_test_rmse:<12.4f} {arima_test_mae:<12.4f} {arima_test_mape:<12.2f} {arima_dir_acc:<12.2f}")
print(f"{'LSTM':<15} {lstm_test_rmse:<12.4f} {lstm_test_mae:<12.4f} {lstm_test_mape:<12.2f} {lstm_test_dir_acc:<12.2f}")
print(f"{'GRU':<15} {test_rmse:<12.4f} {test_mae:<12.4f} {test_mape:<12.2f} {test_dir_acc:<12.2f}")
print("="*80)

In [ ]:
# Cell 71: Save GRU predictions and metrics
gru_results = pd.DataFrame({
    'date': test_seq_dates,
    'actual': y_test_actual,
    'gru_pred': y_test_pred
})
gru_results.to_csv(os.path.join(PROCESSED_DATA_PATH, "gru_test_results.csv"), index=False)

# Save metrics
gru_metrics_dict = {
    'model': 'GRU',
    'val_rmse': val_rmse,
    'val_mae': val_mae,
    'val_mape': val_mape,
    'val_dir_acc': val_dir_acc,
    'test_rmse': test_rmse,
    'test_mae': test_mae,
    'test_mape': test_mape,
    'test_dir_acc': test_dir_acc
}
gru_metrics_df = pd.DataFrame([gru_metrics_dict])
gru_metrics_df.to_csv(os.path.join(PROCESSED_DATA_PATH, "gru_metrics.csv"), index=False)

logger.info("GRU results saved.")

In [ ]:
# Cell 72: (Optional) Quick analysis of GRU vs LSTM
improvement_rmse = (lstm_test_rmse - test_rmse) / lstm_test_rmse * 100
improvement_dir = (test_dir_acc - lstm_test_dir_acc)

print("\nGRU vs LSTM on Test Set:")
print(f"RMSE improvement: {improvement_rmse:+.2f}% (negative means GRU better)")
print(f"Directional accuracy change: {improvement_dir:+.2f} percentage points")

logger.info("Phase 6 complete. GRU model trained and evaluated.")

## CNN-BiGRU Hybrid Model

In [ ]:
# Cell 73: Import necessary libraries (if not already)
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Bidirectional, GRU, Dense, Dropout, Input, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import os
import pickle

logger.info("Starting Phase 7: CNN-BiGRU Hybrid Model")

In [ ]:
# Cell 74: Load preprocessed sequences and targets
PROCESSED_DATA_PATH = "data/processed"

X_train_seq = np.load(os.path.join(PROCESSED_DATA_PATH, "X_train_seq.npy"))
y_train_seq = np.load(os.path.join(PROCESSED_DATA_PATH, "y_train_seq.npy"))
X_val_seq = np.load(os.path.join(PROCESSED_DATA_PATH, "X_val_seq.npy"))
y_val_seq = np.load(os.path.join(PROCESSED_DATA_PATH, "y_val_seq.npy"))
X_test_seq = np.load(os.path.join(PROCESSED_DATA_PATH, "X_test_seq.npy"))
y_test_seq = np.load(os.path.join(PROCESSED_DATA_PATH, "y_test_seq.npy"))

train_seq_dates = np.load(os.path.join(PROCESSED_DATA_PATH, "train_seq_dates.npy"), allow_pickle=True)
val_seq_dates = np.load(os.path.join(PROCESSED_DATA_PATH, "val_seq_dates.npy"), allow_pickle=True)
test_seq_dates = np.load(os.path.join(PROCESSED_DATA_PATH, "test_seq_dates.npy"), allow_pickle=True)

print(f"X_train_seq shape: {X_train_seq.shape}")
print(f"y_train_seq shape: {y_train_seq.shape}")
print(f"X_val_seq shape: {X_val_seq.shape}")
print(f"X_test_seq shape: {X_test_seq.shape}")

In [ ]:
# Cell 75: Load target scaler and scale targets
target_scaler = joblib.load(os.path.join(PROCESSED_DATA_PATH, "target_scaler.pkl"))

y_train_scaled = target_scaler.transform(y_train_seq.reshape(-1, 1)).flatten()
y_val_scaled = target_scaler.transform(y_val_seq.reshape(-1, 1)).flatten()
y_test_scaled = target_scaler.transform(y_test_seq.reshape(-1, 1)).flatten()

In [ ]:
# Cell 76: Define CNN-BiGRU model architecture
def build_cnn_bigru_model(input_shape, 
                          conv_filters=64, 
                          kernel_size=3, 
                          pool_size=2, 
                          gru_units=50, 
                          dropout_rate=0.3, 
                          learning_rate=0.001):
    """
    Build a hybrid CNN-Bidirectional GRU model.
    
    Args:
        input_shape: (window_size, n_features)
        conv_filters: number of filters in Conv1D
        kernel_size: size of convolutional kernel
        pool_size: pooling window size
        gru_units: number of GRU units in each direction
        dropout_rate: dropout rate after each layer
        learning_rate: Adam learning rate
    """
    model = Sequential()
    model.add(Input(shape=input_shape))
    
    # 1D Convolutional layer for feature extraction
    model.add(Conv1D(filters=conv_filters, kernel_size=kernel_size, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Dropout(dropout_rate))
    
    # Bidirectional GRU layer
    model.add(Bidirectional(GRU(units=gru_units, return_sequences=False)))
    model.add(Dropout(dropout_rate))
    
    # Dense layers
    model.add(Dense(25, activation='relu'))
    model.add(Dropout(dropout_rate/2))
    model.add(Dense(1))  # Linear activation for regression
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

# Model parameters (can be tuned)
input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])
conv_filters = 64
kernel_size = 3
pool_size = 2
gru_units = 50
dropout_rate = 0.3
learning_rate = 0.001

model = build_cnn_bigru_model(input_shape, conv_filters, kernel_size, pool_size, gru_units, dropout_rate, learning_rate)
model.summary()

In [ ]:
# Cell 77: Set up callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1),
    ModelCheckpoint(
        filepath=os.path.join("models", "cnn_bigru_best.keras"),
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)
]

logger.info("Callbacks configured.")

In [ ]:
# Cell 78: Train the CNN-BiGRU model
history = model.fit(
    X_train_seq, y_train_scaled,
    validation_data=(X_val_seq, y_val_scaled),
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

# Save training history
with open(os.path.join("models", "cnn_bigru_history.pkl"), 'wb') as f:
    pickle.dump(history.history, f)

logger.info("CNN-BiGRU training completed.")

In [ ]:
# Cell 79: Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Loss
axes[0].plot(history.history['loss'], label='Train Loss')
axes[0].plot(history.history['val_loss'], label='Val Loss')
axes[0].set_title('CNN-BiGRU Model Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss (MSE)')
axes[0].legend()
axes[0].grid(True)

# MAE
axes[1].plot(history.history['mae'], label='Train MAE')
axes[1].plot(history.history['val_mae'], label='Val MAE')
axes[1].set_title('CNN-BiGRU Model MAE')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MAE')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig(os.path.join(PROCESSED_DATA_PATH, "cnn_bigru_training_history.png"))
plt.show()

In [ ]:
# Cell 80: Load the best CNN-BiGRU model
best_model = load_model(os.path.join("models", "cnn_bigru_best.keras"))
logger.info("Best CNN-BiGRU model loaded.")

In [ ]:
# Cell 81: Evaluate on validation and test sets
# Validation predictions
y_val_pred_scaled = best_model.predict(X_val_seq, verbose=0)
y_val_pred = target_scaler.inverse_transform(y_val_pred_scaled).flatten()
y_val_actual = y_val_seq

# Test predictions
y_test_pred_scaled = best_model.predict(X_test_seq, verbose=0)
y_test_pred = target_scaler.inverse_transform(y_test_pred_scaled).flatten()
y_test_actual = y_test_seq

# Compute metrics
def compute_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    mape = np.mean(np.abs((actual - pred) / actual)) * 100
    direction_actual = (actual[1:] > actual[:-1]).astype(int)
    direction_pred = (pred[1:] > actual[:-1]).astype(int)
    dir_acc = np.mean(direction_pred == direction_actual) * 100
    return rmse, mae, mape, dir_acc

val_rmse, val_mae, val_mape, val_dir_acc = compute_metrics(y_val_actual, y_val_pred)
test_rmse, test_mae, test_mape, test_dir_acc = compute_metrics(y_test_actual, y_test_pred)

print("\nCNN-BiGRU Validation Set Performance:")
print(f"RMSE: {val_rmse:.4f}   MAE: {val_mae:.4f}   MAPE: {val_mape:.2f}%   DirAcc: {val_dir_acc:.2f}%")

print("\nCNN-BiGRU Test Set Performance:")
print(f"RMSE: {test_rmse:.4f}   MAE: {test_mae:.4f}   MAPE: {test_mape:.2f}%   DirAcc: {test_dir_acc:.2f}%")

In [ ]:
# Cell 82: Plot CNN-BiGRU predictions vs actual
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Validation
axes[0].plot(val_seq_dates, y_val_actual, label='Actual', color='blue')
axes[0].plot(val_seq_dates, y_val_pred, label='CNN-BiGRU Predicted', color='red', linestyle='--')
axes[0].set_title('CNN-BiGRU Predictions – Validation Set')
axes[0].set_ylabel('Price ($)')
axes[0].legend()
axes[0].grid(True)

# Test
axes[1].plot(test_seq_dates, y_test_actual, label='Actual', color='blue')
axes[1].plot(test_seq_dates, y_test_pred, label='CNN-BiGRU Predicted', color='red', linestyle='--')
axes[1].set_title('CNN-BiGRU Predictions – Test Set')
axes[1].set_ylabel('Price ($)')
axes[1].set_xlabel('Date')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig(os.path.join(PROCESSED_DATA_PATH, "cnn_bigru_predictions.png"))
plt.show()

In [ ]:
# Cell 83: Compare with previous models
# Load previous metrics
arima_results = pd.read_csv(os.path.join(PROCESSED_DATA_PATH, "arima_test_results.csv"))
arima_test_rmse = np.sqrt(mean_squared_error(arima_results['actual'], arima_results['arima_pred']))
arima_test_mae = mean_absolute_error(arima_results['actual'], arima_results['arima_pred'])
arima_test_mape = np.mean(np.abs((arima_results['actual'] - arima_results['arima_pred']) / arima_results['actual'])) * 100
arima_dir_acc = arima_results['direction_pred'].iloc[:-1].eq(arima_results['direction_actual'].iloc[:-1]).mean() * 100

lstm_metrics = pd.read_csv(os.path.join(PROCESSED_DATA_PATH, "lstm_metrics.csv"))
lstm_test_rmse = lstm_metrics['test_rmse'].values[0]
lstm_test_mae = lstm_metrics['test_mae'].values[0]
lstm_test_mape = lstm_metrics['test_mape'].values[0]
lstm_test_dir_acc = lstm_metrics['test_dir_acc'].values[0]

gru_metrics = pd.read_csv(os.path.join(PROCESSED_DATA_PATH, "gru_metrics.csv"))
gru_test_rmse = gru_metrics['test_rmse'].values[0]
gru_test_mae = gru_metrics['test_mae'].values[0]
gru_test_mape = gru_metrics['test_mape'].values[0]
gru_test_dir_acc = gru_metrics['test_dir_acc'].values[0]

print("\n" + "="*90)
print("MODEL COMPARISON ON TEST SET")
print("="*90)
print(f"{'Model':<18} {'RMSE':<12} {'MAE':<12} {'MAPE(%)':<12} {'DirAcc(%)':<12}")
print("-"*90)
print(f"{'ARIMA':<18} {arima_test_rmse:<12.4f} {arima_test_mae:<12.4f} {arima_test_mape:<12.2f} {arima_dir_acc:<12.2f}")
print(f"{'LSTM':<18} {lstm_test_rmse:<12.4f} {lstm_test_mae:<12.4f} {lstm_test_mape:<12.2f} {lstm_test_dir_acc:<12.2f}")
print(f"{'GRU':<18} {gru_test_rmse:<12.4f} {gru_test_mae:<12.4f} {gru_test_mape:<12.2f} {gru_test_dir_acc:<12.2f}")
print(f"{'CNN-BiGRU':<18} {test_rmse:<12.4f} {test_mae:<12.4f} {test_mape:<12.2f} {test_dir_acc:<12.2f}")
print("="*90)

In [ ]:
# Cell 84: Save CNN-BiGRU predictions and metrics
cnn_bigru_results = pd.DataFrame({
    'date': test_seq_dates,
    'actual': y_test_actual,
    'cnn_bigru_pred': y_test_pred
})
cnn_bigru_results.to_csv(os.path.join(PROCESSED_DATA_PATH, "cnn_bigru_test_results.csv"), index=False)

# Save metrics
cnn_bigru_metrics_dict = {
    'model': 'CNN-BiGRU',
    'val_rmse': val_rmse,
    'val_mae': val_mae,
    'val_mape': val_mape,
    'val_dir_acc': val_dir_acc,
    'test_rmse': test_rmse,
    'test_mae': test_mae,
    'test_mape': test_mape,
    'test_dir_acc': test_dir_acc
}
cnn_bigru_metrics_df = pd.DataFrame([cnn_bigru_metrics_dict])
cnn_bigru_metrics_df.to_csv(os.path.join(PROCESSED_DATA_PATH, "cnn_bigru_metrics.csv"), index=False)

logger.info("CNN-BiGRU results saved.")

In [ ]:
# Cell 85: (Optional) Analyze feature importance via integrated gradients or permutation importance
# This is a more advanced step, could be added later.

logger.info("Phase 7 complete. CNN-BiGRU hybrid model trained and evaluated.")

## Transformer for Time Series

In [ ]:
# Cell 86: Import necessary libraries (if not already)
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (Input, Dense, Dropout, LayerNormalization, 
                                     MultiHeadAttention, GlobalAveragePooling1D, 
                                     Add, Flatten)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import os
import pickle

logger.info("Starting Phase 8: Transformer Model")

In [ ]:
# Cell 87: Load preprocessed sequences and targets
PROCESSED_DATA_PATH = "data/processed"

X_train_seq = np.load(os.path.join(PROCESSED_DATA_PATH, "X_train_seq.npy"))
y_train_seq = np.load(os.path.join(PROCESSED_DATA_PATH, "y_train_seq.npy"))
X_val_seq = np.load(os.path.join(PROCESSED_DATA_PATH, "X_val_seq.npy"))
y_val_seq = np.load(os.path.join(PROCESSED_DATA_PATH, "y_val_seq.npy"))
X_test_seq = np.load(os.path.join(PROCESSED_DATA_PATH, "X_test_seq.npy"))
y_test_seq = np.load(os.path.join(PROCESSED_DATA_PATH, "y_test_seq.npy"))

train_seq_dates = np.load(os.path.join(PROCESSED_DATA_PATH, "train_seq_dates.npy"), allow_pickle=True)
val_seq_dates = np.load(os.path.join(PROCESSED_DATA_PATH, "val_seq_dates.npy"), allow_pickle=True)
test_seq_dates = np.load(os.path.join(PROCESSED_DATA_PATH, "test_seq_dates.npy"), allow_pickle=True)

print(f"X_train_seq shape: {X_train_seq.shape}")
print(f"y_train_seq shape: {y_train_seq.shape}")
print(f"X_val_seq shape: {X_val_seq.shape}")
print(f"X_test_seq shape: {X_test_seq.shape}")

In [ ]:
# Cell 88: Load target scaler and scale targets
target_scaler = joblib.load(os.path.join(PROCESSED_DATA_PATH, "target_scaler.pkl"))

y_train_scaled = target_scaler.transform(y_train_seq.reshape(-1, 1)).flatten()
y_val_scaled = target_scaler.transform(y_val_seq.reshape(-1, 1)).flatten()
y_test_scaled = target_scaler.transform(y_test_seq.reshape(-1, 1)).flatten()

In [ ]:
# Cell 89: Define Transformer Encoder Layer
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.1):
    """
    A single transformer encoder block.
    
    Args:
        inputs: Input tensor
        head_size: Dimension of each attention head
        num_heads: Number of attention heads
        ff_dim: Hidden layer size in feed-forward network
        dropout: Dropout rate
    """
    # Multi-head self-attention
    attention = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    attention = Dropout(dropout)(attention)
    attention = LayerNormalization(epsilon=1e-6)(inputs + attention)  # Add & Norm
    
    # Feed-forward network
    ff = Dense(ff_dim, activation="relu")(attention)
    ff = Dropout(dropout)(ff)
    ff = Dense(inputs.shape[-1])(ff)  # Project back to original dimension
    ff = Dropout(dropout)(ff)
    outputs = LayerNormalization(epsilon=1e-6)(attention + ff)  # Add & Norm
    
    return outputs

def build_transformer_model(input_shape, head_size=64, num_heads=4, ff_dim=128, num_layers=2, dropout=0.1, learning_rate=0.001):
    """
    Build a Transformer encoder for time series regression.
    
    Args:
        input_shape: (sequence_length, n_features)
        head_size: Dimension of each attention head
        num_heads: Number of attention heads
        ff_dim: Hidden layer size in feed-forward network
        num_layers: Number of transformer encoder blocks
        dropout: Dropout rate
        learning_rate: Adam learning rate
    """
    inputs = Input(shape=input_shape)
    
    # Initial projection (optional) – could add a Dense layer to increase dimension
    x = inputs
    
    # Stack transformer encoder blocks
    for _ in range(num_layers):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)
    
    # Global pooling to get a fixed-size output
    x = GlobalAveragePooling1D()(x)
    x = Dropout(dropout)(x)
    
    # Final regression head
    x = Dense(50, activation="relu")(x)
    x = Dropout(dropout/2)(x)
    outputs = Dense(1)(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

# Model parameters (can be tuned)
input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])
head_size = 64
num_heads = 4
ff_dim = 128
num_layers = 2
dropout = 0.1
learning_rate = 0.001

model = build_transformer_model(input_shape, head_size, num_heads, ff_dim, num_layers, dropout, learning_rate)
model.summary()

In [ ]:
# Cell 90: Set up callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1),
    ModelCheckpoint(
        filepath=os.path.join("models", "transformer_best.keras"),
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)
]

logger.info("Callbacks configured.")

In [ ]:
# Cell 91: Train the Transformer model
history = model.fit(
    X_train_seq, y_train_scaled,
    validation_data=(X_val_seq, y_val_scaled),
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

# Save training history
with open(os.path.join("models", "transformer_history.pkl"), 'wb') as f:
    pickle.dump(history.history, f)

logger.info("Transformer training completed.")

In [ ]:
# Cell 92: Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Loss
axes[0].plot(history.history['loss'], label='Train Loss')
axes[0].plot(history.history['val_loss'], label='Val Loss')
axes[0].set_title('Transformer Model Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss (MSE)')
axes[0].legend()
axes[0].grid(True)

# MAE
axes[1].plot(history.history['mae'], label='Train MAE')
axes[1].plot(history.history['val_mae'], label='Val MAE')
axes[1].set_title('Transformer Model MAE')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MAE')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig(os.path.join(PROCESSED_DATA_PATH, "transformer_training_history.png"))
plt.show()

In [ ]:
# Cell 93: Load the best Transformer model
best_model = load_model(os.path.join("models", "transformer_best.keras"))
logger.info("Best Transformer model loaded.")

In [ ]:
# Cell 94: Evaluate on validation and test sets
# Validation predictions
y_val_pred_scaled = best_model.predict(X_val_seq, verbose=0)
y_val_pred = target_scaler.inverse_transform(y_val_pred_scaled).flatten()
y_val_actual = y_val_seq

# Test predictions
y_test_pred_scaled = best_model.predict(X_test_seq, verbose=0)
y_test_pred = target_scaler.inverse_transform(y_test_pred_scaled).flatten()
y_test_actual = y_test_seq

# Compute metrics
def compute_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    mape = np.mean(np.abs((actual - pred) / actual)) * 100
    direction_actual = (actual[1:] > actual[:-1]).astype(int)
    direction_pred = (pred[1:] > actual[:-1]).astype(int)
    dir_acc = np.mean(direction_pred == direction_actual) * 100
    return rmse, mae, mape, dir_acc

val_rmse, val_mae, val_mape, val_dir_acc = compute_metrics(y_val_actual, y_val_pred)
test_rmse, test_mae, test_mape, test_dir_acc = compute_metrics(y_test_actual, y_test_pred)

print("\nTransformer Validation Set Performance:")
print(f"RMSE: {val_rmse:.4f}   MAE: {val_mae:.4f}   MAPE: {val_mape:.2f}%   DirAcc: {val_dir_acc:.2f}%")

print("\nTransformer Test Set Performance:")
print(f"RMSE: {test_rmse:.4f}   MAE: {test_mae:.4f}   MAPE: {test_mape:.2f}%   DirAcc: {test_dir_acc:.2f}%")

In [ ]:
# Cell 95: Plot Transformer predictions vs actual
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Validation
axes[0].plot(val_seq_dates, y_val_actual, label='Actual', color='blue')
axes[0].plot(val_seq_dates, y_val_pred, label='Transformer Predicted', color='red', linestyle='--')
axes[0].set_title('Transformer Predictions – Validation Set')
axes[0].set_ylabel('Price ($)')
axes[0].legend()
axes[0].grid(True)

# Test
axes[1].plot(test_seq_dates, y_test_actual, label='Actual', color='blue')
axes[1].plot(test_seq_dates, y_test_pred, label='Transformer Predicted', color='red', linestyle='--')
axes[1].set_title('Transformer Predictions – Test Set')
axes[1].set_ylabel('Price ($)')
axes[1].set_xlabel('Date')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig(os.path.join(PROCESSED_DATA_PATH, "transformer_predictions.png"))
plt.show()

In [ ]:
# Cell 96: Compare with previous models
# Load ARIMA results
arima_results = pd.read_csv(os.path.join(PROCESSED_DATA_PATH, "arima_test_results.csv"))
arima_test_rmse = np.sqrt(mean_squared_error(arima_results['actual'], arima_results['arima_pred']))
arima_test_mae = mean_absolute_error(arima_results['actual'], arima_results['arima_pred'])
arima_test_mape = np.mean(np.abs((arima_results['actual'] - arima_results['arima_pred']) / arima_results['actual'])) * 100
arima_dir_acc = arima_results['direction_pred'].iloc[:-1].eq(arima_results['direction_actual'].iloc[:-1]).mean() * 100

# Load other metrics
lstm_metrics = pd.read_csv(os.path.join(PROCESSED_DATA_PATH, "lstm_metrics.csv"))
gru_metrics = pd.read_csv(os.path.join(PROCESSED_DATA_PATH, "gru_metrics.csv"))
cnn_bigru_metrics = pd.read_csv(os.path.join(PROCESSED_DATA_PATH, "cnn_bigru_metrics.csv"))

print("\n" + "="*100)
print("FINAL MODEL COMPARISON ON TEST SET")
print("="*100)
print(f"{'Model':<20} {'RMSE':<12} {'MAE':<12} {'MAPE(%)':<12} {'DirAcc(%)':<12}")
print("-"*100)
print(f"{'ARIMA':<20} {arima_test_rmse:<12.4f} {arima_test_mae:<12.4f} {arima_test_mape:<12.2f} {arima_dir_acc:<12.2f}")
print(f"{'LSTM':<20} {lstm_metrics['test_rmse'].values[0]:<12.4f} {lstm_metrics['test_mae'].values[0]:<12.4f} {lstm_metrics['test_mape'].values[0]:<12.2f} {lstm_metrics['test_dir_acc'].values[0]:<12.2f}")
print(f"{'GRU':<20} {gru_metrics['test_rmse'].values[0]:<12.4f} {gru_metrics['test_mae'].values[0]:<12.4f} {gru_metrics['test_mape'].values[0]:<12.2f} {gru_metrics['test_dir_acc'].values[0]:<12.2f}")
print(f"{'CNN-BiGRU':<20} {cnn_bigru_metrics['test_rmse'].values[0]:<12.4f} {cnn_bigru_metrics['test_mae'].values[0]:<12.4f} {cnn_bigru_metrics['test_mape'].values[0]:<12.2f} {cnn_bigru_metrics['test_dir_acc'].values[0]:<12.2f}")
print(f"{'Transformer':<20} {test_rmse:<12.4f} {test_mae:<12.4f} {test_mape:<12.2f} {test_dir_acc:<12.2f}")
print("="*100)

In [ ]:
# Cell 97: Save Transformer predictions and metrics
transformer_results = pd.DataFrame({
    'date': test_seq_dates,
    'actual': y_test_actual,
    'transformer_pred': y_test_pred
})
transformer_results.to_csv(os.path.join(PROCESSED_DATA_PATH, "transformer_test_results.csv"), index=False)

# Save metrics
transformer_metrics_dict = {
    'model': 'Transformer',
    'val_rmse': val_rmse,
    'val_mae': val_mae,
    'val_mape': val_mape,
    'val_dir_acc': val_dir_acc,
    'test_rmse': test_rmse,
    'test_mae': test_mae,
    'test_mape': test_mape,
    'test_dir_acc': test_dir_acc
}
transformer_metrics_df = pd.DataFrame([transformer_metrics_dict])
transformer_metrics_df.to_csv(os.path.join(PROCESSED_DATA_PATH, "transformer_metrics.csv"), index=False)

logger.info("Transformer results saved.")

In [ ]:
# Cell 98: (Optional) Visualize attention weights for a sample
# This is an advanced diagnostic; can be added later.

logger.info("Phase 8 complete. Transformer model trained and evaluated.")

## Ensemble Model

In [ ]:
# Cell 99: Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os

logger.info("Starting Phase 9: Ensemble Model")

In [ ]:
# Cell 100: Load test dates and actual values
PROCESSED_DATA_PATH = "data/processed"
test_seq_dates = np.load(os.path.join(PROCESSED_DATA_PATH, "test_seq_dates.npy"), allow_pickle=True)
y_test_actual = np.load(os.path.join(PROCESSED_DATA_PATH, "y_test_seq.npy"))

# Load predictions from each model (assuming saved CSV files exist)
# If not, you can regenerate them from the saved models, but we'll use the saved CSV for simplicity.

models = ['lstm', 'gru', 'cnn_bigru', 'transformer']
predictions = {}
for model_name in models:
    file_path = os.path.join(PROCESSED_DATA_PATH, f"{model_name}_test_results.csv")
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        # Ensure alignment by date (if needed, but we trust they are in same order)
        predictions[model_name] = df[f"{model_name}_pred"].values
        logger.info(f"Loaded {model_name} predictions, shape: {predictions[model_name].shape}")
    else:
        logger.warning(f"Predictions file for {model_name} not found. Skipping.")

# Also load ARIMA for comparison (optional)
arima_path = os.path.join(PROCESSED_DATA_PATH, "arima_test_results.csv")
if os.path.exists(arima_path):
    arima_df = pd.read_csv(arima_path)
    # ARIMA predictions might be shorter if walk-forward started later; we need to align.
    # For simplicity, we'll keep only models with full length.
    # Check lengths
    for name, pred in predictions.items():
        print(f"{name}: {len(pred)}")
    print(f"Actual: {len(y_test_actual)}")

In [ ]:
# Cell 101: Ensure all predictions have the same length
# Some models might have slightly different lengths due to sequence creation offsets.
# We'll truncate to the minimum common length.
lengths = [len(y_test_actual)] + [len(pred) for pred in predictions.values()]
min_len = min(lengths)
if min_len < len(y_test_actual):
    logger.info(f"Truncating all arrays to minimum length: {min_len}")
    y_test_actual = y_test_actual[:min_len]
    for name in predictions:
        predictions[name] = predictions[name][:min_len]
    test_seq_dates = test_seq_dates[:min_len]

print(f"Final shapes: actual {y_test_actual.shape}, predictions all {len(predictions['lstm'])}")

In [ ]:
# Cell 102: Simple average ensemble
# Stack predictions into a 2D array (n_samples, n_models)
pred_array = np.column_stack([predictions[name] for name in models])
ensemble_simple_avg = np.mean(pred_array, axis=1)

# Compute metrics for simple average
def compute_metrics(actual, pred, name="Ensemble"):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    mape = np.mean(np.abs((actual - pred) / actual)) * 100
    direction_actual = (actual[1:] > actual[:-1]).astype(int)
    direction_pred = (pred[1:] > actual[:-1]).astype(int)
    dir_acc = np.mean(direction_pred == direction_actual) * 100
    print(f"\n{name} Performance:")
    print(f"RMSE: {rmse:.4f}   MAE: {mae:.4f}   MAPE: {mape:.2f}%   DirAcc: {dir_acc:.2f}%")
    return rmse, mae, mape, dir_acc

simple_metrics = compute_metrics(y_test_actual, ensemble_simple_avg, "Simple Average Ensemble")

In [ ]:
# Cell 103: Weighted average ensemble based on validation performance
# Load validation metrics for each model
val_metrics = {}
for model_name in models:
    metrics_file = os.path.join(PROCESSED_DATA_PATH, f"{model_name}_metrics.csv")
    if os.path.exists(metrics_file):
        df = pd.read_csv(metrics_file)
        # Use validation RMSE (lower is better) to compute weights
        # We'll use inverse RMSE as weight
        val_rmse = df['val_rmse'].values[0]
        val_metrics[model_name] = val_rmse
    else:
        logger.warning(f"Metrics file for {model_name} not found. Using equal weight.")

if val_metrics:
    # Compute weights: inverse of RMSE, normalized
    inv_rmse = {name: 1/rmse for name, rmse in val_metrics.items()}
    total = sum(inv_rmse.values())
    weights = {name: inv_rmse[name]/total for name in inv_rmse}
    print("Weights based on validation RMSE:")
    for name, w in weights.items():
        print(f"  {name}: {w:.4f}")
    
    # Apply weighted average
    weighted_pred = np.zeros_like(ensemble_simple_avg)
    for i, name in enumerate(models):
        if name in weights:
            weighted_pred += weights[name] * predictions[name]
    
    weighted_metrics = compute_metrics(y_test_actual, weighted_pred, "Weighted Average Ensemble")
else:
    logger.warning("No validation metrics found, skipping weighted ensemble.")

In [ ]:
# Cell 104: Compare ensemble with individual models
# Load individual model test metrics
individual_metrics = {}
for model_name in models:
    metrics_file = os.path.join(PROCESSED_DATA_PATH, f"{model_name}_metrics.csv")
    if os.path.exists(metrics_file):
        df = pd.read_csv(metrics_file)
        individual_metrics[model_name] = {
            'RMSE': df['test_rmse'].values[0],
            'MAE': df['test_mae'].values[0],
            'MAPE': df['test_mape'].values[0],
            'DirAcc': df['test_dir_acc'].values[0]
        }

# Add ARIMA if available
if os.path.exists(arima_path):
    individual_metrics['ARIMA'] = {
        'RMSE': arima_test_rmse,
        'MAE': arima_test_mae,
        'MAPE': arima_test_mape,
        'DirAcc': arima_dir_acc
    }

# Add ensemble metrics
individual_metrics['Simple_Avg_Ensemble'] = {
    'RMSE': simple_metrics[0],
    'MAE': simple_metrics[1],
    'MAPE': simple_metrics[2],
    'DirAcc': simple_metrics[3]
}
if 'weighted_metrics' in locals():
    individual_metrics['Weighted_Ensemble'] = {
        'RMSE': weighted_metrics[0],
        'MAE': weighted_metrics[1],
        'MAPE': weighted_metrics[2],
        'DirAcc': weighted_metrics[3]
    }

# Create comparison table
comparison_df = pd.DataFrame(individual_metrics).T
comparison_df = comparison_df.round(4)
print("\n" + "="*100)
print("FINAL ENSEMBLE COMPARISON")
print("="*100)
print(comparison_df.to_string())
print("="*100)

# Save comparison
comparison_df.to_csv(os.path.join(PROCESSED_DATA_PATH, "ensemble_comparison.csv"))

In [ ]:
# Cell 105: Plot ensemble predictions vs actual
plt.figure(figsize=(16, 6))
plt.plot(test_seq_dates, y_test_actual, label='Actual', color='blue', linewidth=1.5)
plt.plot(test_seq_dates, ensemble_simple_avg, label='Simple Average Ensemble', color='red', linestyle='--', linewidth=1.5)
if 'weighted_pred' in locals():
    plt.plot(test_seq_dates, weighted_pred, label='Weighted Ensemble', color='green', linestyle=':', linewidth=1.5)
plt.title('Ensemble Predictions vs Actual - Test Set')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(PROCESSED_DATA_PATH, "ensemble_predictions.png"))
plt.show()

In [ ]:
# Cell 106: Save ensemble predictions
ensemble_results = pd.DataFrame({
    'date': test_seq_dates,
    'actual': y_test_actual,
    'simple_avg_ensemble': ensemble_simple_avg
})
if 'weighted_pred' in locals():
    ensemble_results['weighted_ensemble'] = weighted_pred
ensemble_results.to_csv(os.path.join(PROCESSED_DATA_PATH, "ensemble_test_results.csv"), index=False)

logger.info("Phase 9 complete. Ensemble model evaluated.")